In [1]:
import sys
import os
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

import sys 
sys.path.append('/Users/gracecolverd/New_dataset')
from ml_utils.src.model_col_settings import region_mapping, settings_col_dict_new


/Users/gracecolverd/miniconda3/envs/ag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:

data_path = '/Volumes/T9/Data_downloads/new-data-outputs/ml_input/Lincolnshire_fuel_porveryt_data.csv'
output_path ='/Users/gracecolverd/New_dataset/ml_scripts/proc'
model_preset= 'medium_quality'
time_limit = 1000
train_subset_prop = 1
model_types = 'all'
target = 'totalgas'
column_setting_list  = [24, 25,26,27]
run_regionally = False 

label = 'total_gas'

def transform(df, label, col_setting):
    cols = settings_col_dict_new[col_setting]
    working_cols = cols + [label]
    df = df[working_cols]
    df = df[~df[label].isna()]
    return df


In [11]:
def save_results(results, output_path):
    res_string = str(results)
    # summary = predictor.fit_summary()
    with open(os.path.join(output_path, 'model_summary.txt'), 'w') as f:
        f.write(res_string)



In [14]:
def run_local(name, column_setting):
    output_directory = os.path.join(output_path, f'{name}__colset_{column_setting}')
    df = pd.read_csv(data_path)
    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
    train_data = transform(TabularDataset(train_data), label, column_setting )


    # Reduce the training dataset if needed
    if train_subset_prop != 1:
        train_subset, _ = train_test_split(train_data, test_size=1-train_subset_prop, random_state=42)
        # train_subset.to_csv(os.path.join(output_directory, 'train_subset.csv'), index=False)
    else:
        train_subset = train_data  



    excl_models=[]

    size_train = len(train_subset) 
    predictor = TabularPredictor(label, path=output_directory).fit(train_subset, 
                                                                time_limit=time_limit,
                                                                presets=model_preset,
                                                                excluded_model_types=excl_models)


    test_data = transform(TabularDataset(test_data), label, column_setting)
    # test_data.to_csv(os.path.join(output_directory, 'test_data_dummy.csv'), index=False)
    y_pred = predictor.predict(test_data.drop(columns=[label]))
    results = predictor.evaluate_predictions(y_true=test_data[label], y_pred=y_pred, auxiliary_metrics=True)
    size_test = len(test_data)

    print(results)

    save_results(results, output_directory)
    res = predictor.leaderboard(test_data)
    res.to_csv(os.path.join(output_directory, 'leaderboard_results.csv'))

    res = predictor.leaderboard(test_data)
    print(res)
        

In [15]:
for cl in column_setting_list:
    print(cl)
    run_local('fuel_poc', cl)
    print('done')

Presets specified: ['medium_quality']
Beginning AutoGluon training ... Time limit = 1000s
AutoGluon will save models to "/Users/gracecolverd/New_dataset/ml_scripts/proc/fuel_poc__colset_24"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 22.1.0: Sun Oct  9 20:14:30 PDT 2022; root:xnu-8792.41.9~2/RELEASE_ARM64_T8103
CPU Count:          8
Memory Avail:       4.63 GB / 16.00 GB (28.9%)
Disk Space Avail:   44.63 GB / 460.43 GB (9.7%)
Train Data Rows:    5374
Train Data Columns: 70
Label Column:       total_gas
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (950408.2484057454, 19456.860758487797, 208880.85354, 141225.70553)
	If 'regression' is not the correct problem_type, please manually specify the problem_type 

24


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Useless Original Features (Count: 6): ['CDD_winter', 'perc_all_res', 'Tall flats 6-15 storeys_pct', 'Unknown_pct', 'Very large detached_pct', 'Very tall point block flats_pct']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Unused Original Features (Count: 12): ['CDD_summer', 'all_res_total_buildings', 'clean_res_base_floor_total', 'clean_res_basement_hea

{'root_mean_squared_error': -42640.84294396354, 'mean_squared_error': -1818241486.9717648, 'mean_absolute_error': -29663.318807106953, 'r2': 0.9180561216388143, 'pearsonr': 0.9582102478476255, 'median_absolute_error': -20423.8480389053}


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-42640.842944,-38788.635271,root_mean_squared_error,0.196707,0.048562,30.226821,0.002699,0.000332,0.010610,2,True,12
1,LightGBMXT,-43544.747384,-39781.926099,root_mean_squared_error,0.023348,0.006402,5.573309,0.023348,0.006402,5.573309,1,True,3
2,NeuralNetFastAI,-43565.448523,-40960.733722,root_mean_squared_error,0.099501,0.017398,9.575653,0.099501,0.017398,9.575653,1,True,8
3,CatBoost,-43793.320208,-39337.786505,root_mean_squared_error,0.021373,0.007812,7.019388,0.021373,0.007812,7.019388,1,True,6
4,LightGBM,-44865.491507,-40421.654182,root_mean_squared_error,0.022983,0.008078,5.132051,0.022983,0.008078,5.132051,1,True,4
5,XGBoost,-44969.624104,-40518.449748,root_mean_squared_error,0.026803,0.008539,2.915811,0.026803,0.008539,2.915811,1,True,9
6,LightGBMLarge,-45173.870714,-42402.517503,root_mean_squared_error,0.031007,0.008745,19.607104,0.031007,0.008745,19.607104,1,True,11
7,ExtraTreesMSE,-45804.639473,-42581.455987,root_mean_squared_error,0.208128,0.082354,2.785001,0.208128,0.082354,2.785001,1,True,7
8,NeuralNetTorch,-45978.509804,-42365.849553,root_mean_squared_error,0.048760,0.042819,11.509005,0.048760,0.042819,11.509005,1,True,10
9,RandomForestMSE,-46390.147782,-43701.962232,root_mean_squared_error,0.160118,0.058806,8.321134,0.160118,0.058806,8.321134,1,True,5


In [ ]:


# pred = predictor.feature_importance(test_data)
# pred.to_csv(os.path.join(output_directory, 'feature_importance.csv'))



